In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from tqdm import tqdm

import multiprocessing
from multiprocessing import Pool

num_cores = multiprocessing.cpu_count()
print(num_cores)

2


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def process_file(file_name):
  with open(file_name, "r") as file:
    text = file.read()
  return text

In [ ]:
def choose_data(n):
  # Choose single movie (Kung Fu Panda) as data
  if n == 1:
    file_names = [ "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Family/kungfupanda_dialog.txt" ]
    data = process_file(file_names[0])
  # Choose 2 movies (Die Hard and Die Hard 2) as data
  if n == 2:
    file_names = ["/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/diehard_dialog.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/diehard2_dialog.txt"]
    data = process_file(file_names[0]) + ' STARTTOKEN ENDTOKEN ' + process_file(file_names[1])

  # Choose 3 movies (all Starwars) as data
  if n == 3:
    file_names = ["/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/starwarsanewhope_dialog.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/starwarstheempirestrikesback_dialog.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/starwarsattackoftheclones_dialog.txt"]
    data = ""
    for file_name in file_names:
      data += ' STARTTOKEN '
      data += process_file(file_name)
      data += ' ENDTOKEN '

  # Choose All Harry Potter Movies (- Deathly Hallows Pt.2)
  if n == 4:
    file_names = ["/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Sorcerer s Stone_0241527.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Chamber of Secrets_0295297.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Prisoner of Azkaban_0304141.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Goblet of Fire_0330373.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Half Blood Prince_0417741.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Order of the Phoenix_0373889.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Deathly Hallows Part 1_0926084.txt"]
    data = ""
    for file_name in file_names:
      data += ' STARTTOKEN '
      data += process_file(file_name)
      data += ' ENDTOKEN '

  return data

In [ ]:
data = choose_data(4)

In [ ]:
import re

def remove_URLs_and_Numbers(text):
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'\d+', '', text)
  return text

In [ ]:
text = remove_URLs_and_Numbers(data)

In [ ]:
tokens = word_tokenize(text)

In [ ]:
print(tokens[:50])

['STARTTOKEN', 'HARRY', 'POTTER', 'AND', 'THE', 'SORCERER', '’', 'S', 'STONE', 'BR', 'eee', 'Screenplay', 'by', 'Steve', 'Kloves', 'Based', 'on', 'the', 'novel', 'by', 'J.K.', 'Rowling', 'Shooting', 'Draft', '//', 'Blue', 'Revision', '//', '-', 'Pink', 'Revision', '//', '.', 'Yellow', 'Revision', '//', 'Green', 'Revision', '//', 'Gold', 'Revision', '//', 'Buff', 'Revision', '//', '‘', 'Salmon', 'Revision', '//', 'Cherry']


In [ ]:
i=0
while i < len(tokens):
  if tokens[i] == 'STARTTOKEN':
    tokens[i] = '<s>'
  if tokens[i] == 'ENDTOKEN':
    tokens[i] = '</s>'
  if tokens[i] == '.':
    tokens[i] = '</s>'
    tokens.insert(i+1, '<s>')
  i += 1

In [ ]:
print(tokens[:50])

['<s>', 'HARRY', 'POTTER', 'AND', 'THE', 'SORCERER', '’', 'S', 'STONE', 'BR', 'eee', 'Screenplay', 'by', 'Steve', 'Kloves', 'Based', 'on', 'the', 'novel', 'by', 'J.K.', 'Rowling', 'Shooting', 'Draft', '//', 'Blue', 'Revision', '//', '-', 'Pink', 'Revision', '//', '</s>', '<s>', 'Yellow', 'Revision', '//', 'Green', 'Revision', '//', 'Gold', 'Revision', '//', 'Buff', 'Revision', '//', '‘', 'Salmon', 'Revision', '//']


In [ ]:
from nltk.tokenize.sonority_sequencing import punctuation
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer

def remove_Stopwords_and_Punctuation_and_Stem(tokens):
  stemmer = SnowballStemmer(language='english')
  englishStopwords = stopwords.words('english')
  puncuation = string.punctuation
  # stemmer.stem(token)
  tokens = [ token.lower() for token in tokens if token not in englishStopwords and token not in punctuation ]
  return tokens

In [ ]:
tokens = remove_Stopwords_and_Punctuation_and_Stem(tokens)

In [ ]:
lookup = {}

for i in range(len(tokens)-1):
  token = tokens[i]
  nextToken = tokens[i+1]
  if token not in lookup.keys():
    lookup[token] = {}
  if nextToken not in lookup[token].keys():
    lookup[token][nextToken] = 0
  lookup[token][nextToken] += 1

In [ ]:
for key in lookup.keys():
  count = 0
  for val in lookup[key]:
    count += lookup[key][val]
  for val in lookup[key]:
    lookup[key][val] /= count

In [ ]:
print(lookup)

In [ ]:
import numpy as np
import numpy.random as rand

def generate_sentences(num_sentences, lookup=lookup):
  sentence = ""
  currentToken = '<s>'
  num_stop_tokens = 0
  while num_stop_tokens < num_sentences:
    probs = []
    keys = []
    for key in lookup[currentToken]:
      keys.append(key)
      probs.append(lookup[currentToken][key])
    i = rand.multinomial(1,probs)
    index = 0
    for j in range(len(i)):
      if i[j] == 1:
        index = j
        break
    currentToken = keys[index]
    sentence = sentence + ' ' + currentToken
    if currentToken == '</s>':
      num_stop_tokens += 1
    
  return sentence


In [ ]:
print(generate_sentences(10))

 he who four come tell i ’ even ca n't think i doin ’ s jet fire muted </s> <s> ron glance lupin there silence hangs ... myrtle well ’ sure two -- hermione please would count three lads </s> <s> vernon comes us things people “ dumbledore 's send people wander lupin harry scrambles stairs wormtail b-blood enemy ... « mane ie enchanted car reappears blinks utterly mad </s> <s> it ’ </s> <s> professor mcgonagall looks regards hermione ron chuckles uneasily </s> <s> dark forest home dobby brightens </s> <s> rev </s> <s> dumbledore passes his robes </s> <s> dudley ‘ don miss granger </s> <s> fred simply shadows emerge </s> <s> continued the quill racing across lake ac int </s>
